In [120]:
import pandas as pd 
from sodapy import Socrata
from pymongo import MongoClient
import time
import datetime
from IPython.display import clear_output

In [78]:
# api use - https://dev.socrata.com/foundry/www.dallasopendata.com/9fxf-t2tr
#
# client = Socrata('www.dallasopendata.com',
#                  'KfIrXaHOglxeS3ELKoScuOmr8',
#                  username="bryce_ka@hotmail.com",
#                  password="Bantho750451580!")

In [114]:
# data is only live so need to save data in a db for analytics 

In [89]:
client = MongoClient()
client.DallasPD_DB
db  = client['DallasPD_DB']
db.drop_collection(running_db)
running_data = db.running_db

In [115]:
# insert data into db 

In [116]:
def pull_data():
    results = client.get("9fxf-t2tr", limit=2000)
    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)
    results_df['date'] = pd.to_datetime(results_df['date'], dayfirst = True)
    return results_df

def data_pull_timer(m):
    # Calculate the total number of seconds
    total_seconds =  m * 60
    total_seconds+=1
    # While loop that checks if total_seconds reaches zero
    # If not zero, decrement total time by one second
    while total_seconds > 0:
        # Timer represents time left on countdown
        timer = datetime.timedelta(seconds = total_seconds)
        # Prints the time left on the timer
        print(timer, end="\r")
        # Delays the program one second
        time.sleep(1)
        # Reduces total time by one second
        total_seconds -= 1
    results_df = pull_data()
    return results_df

In [122]:
client = Socrata('www.dallasopendata.com',
                 'KfIrXaHOglxeS3ELKoScuOmr8',
                 username="bryce_ka@hotmail.com",
                 password="Bantho750451580!")

pulled_data = pull_data()
running_data.insert_many(pulled_data.to_dict('records'))

count = True
while count != False:
    # get current db 
    running = pd.DataFrame(list(running_data.find())).drop('_id', axis = 1)
    print("current number of incidents: ", len(running))
    
    # update the constant db for the application or program 
    # 1. identify new rows using incident number date time unit number 
    updated_df = pd.concat([running, pulled_data], axis=0).drop_duplicates()
    running = pd.DataFrame(list(running_data.find()))
    new_documents = updated_df.merge(running, how='left', on=['incident_number', 'date', 'time', 'unit_number'], suffixes=('', '_y'))
    new_documents = new_documents[new_documents['_id'].isnull()][['incident_number', 'division', 'nature_of_call', 'priority', 'date', 'time', 'unit_number', 'location', 'beat', 'reporting_area', 'status', 'block']]
    print(f"found {len(new_documents)} new incident(s)")
    if len(new_documents)!= 0:
        running_data.insert_many(new_documents.to_dict('records'))
    
    # re initialize client 
    client = Socrata('www.dallasopendata.com',
                 'KfIrXaHOglxeS3ELKoScuOmr8',
                 username="bryce_ka@hotmail.com",
                 password="Bantho750451580!")
    # pull past 2 minutes of data 
    pulled_data = data_pull_timer(2)
    clear_output(wait=False)

current number of incidents:  942
found 0 new incident(s)


KeyboardInterrupt: 